# Dataset Exploration

In [38]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter
from copy import deepcopy
from rich import inspect
from tqdm import tqdm

# torch
import torch

# monai
import monai
from monai.data import ImageDataset, ThreadDataLoader


In [79]:
# rtk
from rtk import repl
from rtk.config import (
    set_hydra_configuration,
    Configuration,
    DatasetConfiguration,
    JobConfiguration,
)
from rtk.utils import login, hydra_instantiate, get_console, get_logger


In [80]:
repl.install(show_locals=False)
console = get_console()
logger = get_logger("rtk.notebook")
monai.config.print_config()

MONAI version: 1.3.0
Numpy version: 1.23.5
Pytorch version: 2.1.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: /home/<username>/anaconda3/envs/rtk/lib/python3.11/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.13
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.22.0
scipy version: 1.11.3
Pillow version: 10.0.1
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.16.0+cu121
tqdm version: 4.66.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 2.1.2
einops version: 0.7.0
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 2.8.0
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    ht

### Constant setup

In [41]:
# configs
config_name = "tests"
config_path = "../configs/"
config_dir = os.path.abspath(config_path)
config_dir

'/home/nicoleg/workspaces/ResearchToolKit/configs'

In [42]:
cfg: Configuration = set_hydra_configuration(
        config_name=config_name,
        init_method_kwargs={"config_dir": config_dir},
        ConfigurationInstance=Configuration,
    )
dataset_cfg: DatasetConfiguration = cfg.datasets
job_cfg: JobConfiguration = cfg.job
monai.utils.set_determinism(seed=job_cfg.random_state)
cfg

[14:33:22] INFO     rtk.config: Creating configuration: 'tests'                                       ]8;id=288389;file:///home/nicoleg/workspaces/ResearchToolKit/rtk/config.py\config.py]8;;\:]8;id=256787;file:///home/nicoleg/workspaces/ResearchToolKit/rtk/config.py#292\292]8;;\
                    [0m                                                                                            

Configuration(
    date='2023-11-06',
    postfix='',
    timestamp='14-33-22',
    datasets={'dim': 229, 'extension': '.png', 'index': 'Image Index', 'labels': ['Non-Pneumonia', 'Pneumonia'], 'target': 'Finding Labels', 'patient_data': '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle_RGB/Data_Entry_2017.csv', 'scan_data': '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle_RGB/', 'encoding': {'Non-Pneumonia': 0, 'Pneumonia': 1}, 'instantiate': {'_target_': 'monai.data.ImageDataset'}, 'dataloader': {'_target_': 'monai.data.ThreadDataLoader', 'batch_size': 16, 'num_workers': 24}, 'preprocessing': {'positive_class': 'Pneumonia', 'subset': ['Atelectasis', 'Edema', 'Effusion', 'Consolidation', 'Pneumonia'], 'use_sampling': True, 'sampling_method': {'sample_to_value': 5000, 'method': {'_partial_': True, '_target_': 'imblearn.datasets.make_imbalance', 'verbose': True}}}, 'transforms': {'load': [{'_target_': 'monai.transforms.ScaleIntensity'}, {'_target_': 'monai.transforms.EnsureChannelFirst'}, {'_target_': 'monai.transforms.EnsureType', 'track_meta': True}, {'_target_': 'monai.transforms.Resize', 'spatial_size': ['${datasets.dim}', '${datasets.dim}']}], 'train': [{'_target_': 'torchvision.transforms.RandomResizedCrop', 'scale': [0.75, 1.25], 'size': ['${datasets.dim}', '${datasets.dim}'], 'antialias': True}, {'_target_': 'torchvision.transforms.RandomAffine', 'degrees': 45, 'shear': 16, 'translate': [0.15, 0.2]}]}},
    job={'device': 1, 'dry_run': True, 'epoch_length': 8, 'max_epochs': 2, 'mode': 'train', 'perform_validation': True, 'random_state': 42, 'set_track_meta': False, 'use_autocast': True, 'use_azureml': True, 'use_mlflow': True, 'use_pretrained': True, 'use_transforms': True, 'train_test_split': {'shuffle': True, 'test_size': 0.1}, 'prepare_function': {'_target_': 'rtk.ignite.prepare_run'}, 'tags': {'type': 'test'}},
    models={'model': {'_target_': 'timm.create_model', 'model_name': 'resnet50', 'num_classes': 2, 'pretrained': '${job.use_pretrained}'}, 'load_model': {'name': 'resnet50_PEDIATRICS_CHESTXRAY14', 'version': 1}, 'criterion': {'_target_': 'torch.nn.CrossEntropyLoss'}, 'optimizer': {'_target_': 'torch.optim.Adam', 'lr': 1e-05, 'weight_decay': 0.01}},
    ignite={'log_interval': 5, 'score_name': 'loss', 'use_checkpoint': True, 'use_early_stopping': True, 'use_lr_scheduler': False, 'use_multi_gpu': False, 'metrics': {'Accuracy': None, 'Loss': None, 'Predictions': None}, 'checkpoint': {'_target_': 'ignite.handlers.checkpoint.Checkpoint', 'save_handler': 'artifacts/checkpoints/', 'filename_prefix': 'best', 'n_saved': 10}, 'early_stopping': {'_target_': 'ignite.handlers.early_stopping.EarlyStopping', 'patience': 5}, 'lr_scheduler': None},
    mlflow={'tracking_uri': 'file:///mlruns/'},
    sklearn={'model_selection': {'train_test_split': {'shuffle': True, 'test_size': 0.1}}}
)

In [43]:
patient_path = dataset_cfg.patient_data
patient_path

'/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle_RGB/Data_Entry_2017.csv'

In [44]:
scan_path = "/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/"
scan_path

'/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/'

In [45]:
# display single image
# depth_slice = scan.shape[2] // 2
# display_scan_slice = scan[:, :, depth_slice].numpy()
# plt.imshow(display_scan_slice, cmap="bone");

# Exploration

In [46]:
# https://stackoverflow.com/questions/339007/how-do-i-pad-a-string-with-zeroes
index = dataset_cfg.index
target = dataset_cfg.target
label_encoding = 1
patient_df = pd.read_csv(patient_path).set_index(index)
label_path = os.path.join(scan_path, f"images_{label_encoding:03}", "images")
os.listdir(label_path)


[
    '00000096_001.png',
    '00001250_005.png',
    '00001220_000.png',
    '00000419_000.png',
    '00000654_001.png',
    '00000827_016.png',
    '00000468_025.png',
    ... +4992
]

In [47]:
# subset to the indicated indices in the label encoding
filename_matches = {"image_files": [], index: []}

for filename in os.listdir(label_path):
    filename_matches[index].append(filename)
    filename_matches["image_files"].append(os.path.join(label_path, filename))

filename_matches


{
    'image_files': [
        '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/images_001/images/00000096_001.png',
        '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/images_001/images/00001250_005.png',
        '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/images_001/images/00001220_000.png',
        '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/images_001/images/00000419_000.png',
        '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/images_001/images/00000654_001.png',
        '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/images_001/images/00000827_016.png',
        '/home/nicoleg/workspaces/dissertation/.data/Chest_XRay_14_Kaggle/images_001/images/00000468_025.png',
        ... +4992
    ],
    'Image Index': [
        '00000096_001.png',
        '00001250_005.png',
        '00001220_000.png',
        '00000419_000.png',
        '00000654_001.png',
        '00000827_016.png',
        '00000468_025.png',
        ... +4992
    ]
}

## Wrangling data

In [48]:
patient_df.head()

,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
Image Index,,,,,,,,,,,
00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [49]:
# drop unnecessary columns

drop_columns = [
    "Follow-up #",
    "OriginalImage[Width",
    "Height]",
    "OriginalImagePixelSpacing[x",
    "y]",
    "Unnamed: 11",
    "View Position",
]
patient_df = patient_df.drop(columns=drop_columns)
patient_df.head()

,Finding Labels,Patient ID,Patient Age,Patient Gender
Image Index,,,,
00000001_000.png,Cardiomegaly,1,58,M
00000001_001.png,Cardiomegaly|Emphysema,1,58,M
00000001_002.png,Cardiomegaly|Effusion,1,58,M
00000002_000.png,No Finding,2,81,M
00000003_000.png,Hernia,3,81,F


In [50]:
def unpack_images():
    """"""

    filename_matches = {"image_files": [], index: []}

    for encoding in range(1, 13):
        label_path = os.path.join(scan_path, f"images_{encoding:03}", "images")

        for filename in os.listdir(label_path):
            filename_matches[index].append(filename)
            filename_matches["image_files"].append(os.path.join(label_path, filename))

    return pd.DataFrame.from_dict(filename_matches, orient="columns").set_index(index)

matches = unpack_images()
len(matches), matches.head()

(
    112120,
                                                            image_files
Image Index                                                        
00000096_001.png  /home/nicoleg/workspaces/dissertation/.data/Ch...
00001250_005.png  /home/nicoleg/workspaces/dissertation/.data/Ch...
00001220_000.png  /home/nicoleg/workspaces/dissertation/.data/Ch...
00000419_000.png  /home/nicoleg/workspaces/dissertation/.data/Ch...
00000654_001.png  /home/nicoleg/workspaces/dissertation/.data/Ch...
)

In [51]:
patient_df = patient_df.merge(matches, on=index, how="inner")
patient_df.head()

,Finding Labels,Patient ID,Patient Age,Patient Gender,image_files
Image Index,,,,,
00000001_000.png,Cardiomegaly,1,58,M,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000001_001.png,Cardiomegaly|Emphysema,1,58,M,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000001_002.png,Cardiomegaly|Effusion,1,58,M,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000002_000.png,No Finding,2,81,M,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000003_000.png,Hernia,3,81,F,/home/nicoleg/workspaces/dissertation/.data/Ch...


In [52]:
with open(os.path.join(scan_path, "train_val_list.txt"), "r") as f:
    train_val_list = f.readlines()

train_val_list

[
    '00000001_000.png\n',
    '00000001_001.png\n',
    '00000001_002.png\n',
    '00000002_000.png\n',
    '00000004_000.png\n',
    '00000005_000.png\n',
    '00000005_001.png\n',
    ... +86517
]

In [53]:
train_val_list = [idx.strip() for idx in train_val_list]
train_val_list

[
    '00000001_000.png',
    '00000001_001.png',
    '00000001_002.png',
    '00000002_000.png',
    '00000004_000.png',
    '00000005_000.png',
    '00000005_001.png',
    ... +86517
]

In [54]:
# do the same for the test list
with open(os.path.join(scan_path, "test_list.txt"), "r") as f:
    test_list = f.readlines()
    test_list = [idx.strip() for idx in test_list]

test_list

[
    '00000003_000.png',
    '00000003_001.png',
    '00000003_002.png',
    '00000003_003.png',
    '00000003_004.png',
    '00000003_005.png',
    '00000003_006.png',
    ... +25589
]

In [55]:
train_df = patient_df[patient_df.index.isin(train_val_list)]
train_df.head()

,Finding Labels,Patient ID,Patient Age,Patient Gender,image_files
Image Index,,,,,
00000001_000.png,Cardiomegaly,1,58,M,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000001_001.png,Cardiomegaly|Emphysema,1,58,M,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000001_002.png,Cardiomegaly|Effusion,1,58,M,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000002_000.png,No Finding,2,81,M,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000004_000.png,Mass|Nodule,4,82,M,/home/nicoleg/workspaces/dissertation/.data/Ch...


In [56]:
test_df = patient_df[patient_df.index.isin(test_list)]
test_df.head()

,Finding Labels,Patient ID,Patient Age,Patient Gender,image_files
Image Index,,,,,
00000003_000.png,Hernia,3,81,F,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000003_001.png,Hernia,3,74,F,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000003_002.png,Hernia,3,75,F,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000003_003.png,Hernia|Infiltration,3,76,F,/home/nicoleg/workspaces/dissertation/.data/Ch...
00000003_004.png,Hernia,3,77,F,/home/nicoleg/workspaces/dissertation/.data/Ch...


## Pneumonia to metaclass

In [57]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

In [58]:
# subsetting
# all_labels = dataset_cfg.all_labels
labels = dataset_cfg.labels
positive_class = dataset_cfg.preprocessing["positive_class"]
console.print(f"Positive class: '{positive_class}'")
console.print(f"New labels:\n{labels}")

Positive class: 'Pneumonia'

New labels:
['Non-Pneumonia', 'Pneumonia']

In [59]:
# unique

console.print("Data description:")
console.print(patient_df[target].nunique())
patient_target_counts = Counter(patient_df[target])
patient_target_counts

Data description:

836

Counter({
    'Cardiomegaly': 1093,
    'Cardiomegaly|Emphysema': 19,
    'Cardiomegaly|Effusion': 484,
    'No Finding': 60361,
    'Hernia': 110,
    'Hernia|Infiltration': 15,
    'Mass|Nodule': 394,
    ... +829
})

In [60]:
temp = list(patient_target_counts.keys())[:10]
for key in temp:
    classes = key.split("|")
    console.print(classes)

['Cardiomegaly']

['Cardiomegaly', 'Emphysema']

['Cardiomegaly', 'Effusion']

['No Finding']

['Hernia']

['Hernia', 'Infiltration']

['Mass', 'Nodule']

['Infiltration']

['Effusion', 'Infiltration']

['Nodule']

In [61]:
multi_target = f"multi_{target}"
patient_df[multi_target] = patient_df[target].apply(lambda x: x.split("|"))
patient_df[multi_target]

Image Index
00000001_000.png               [Cardiomegaly]
00000001_001.png    [Cardiomegaly, Emphysema]
00000001_002.png     [Cardiomegaly, Effusion]
00000002_000.png                 [No Finding]
00000003_000.png                     [Hernia]
                              ...            
00030801_001.png            [Mass, Pneumonia]
00030802_000.png                 [No Finding]
00030803_000.png                 [No Finding]
00030804_000.png                 [No Finding]
00030805_000.png                 [No Finding]
Name: multi_Finding Labels, Length: 112120, dtype: object

In [62]:
mlb.fit(patient_df[multi_target])
mlb.classes_

array(['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
       'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration',
       'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia',
       'Pneumothorax'], dtype=object)

In [63]:
all_class_encoding = {label: i for i, label in enumerate(mlb.classes_)}
console.print(all_class_encoding)

{
    'Atelectasis': 0,
    'Cardiomegaly': 1,
    'Consolidation': 2,
    'Edema': 3,
    'Effusion': 4,
    'Emphysema': 5,
    'Fibrosis': 6,
    'Hernia': 7,
    'Infiltration': 8,
    'Mass': 9,
    'No Finding': 10,
    'Nodule': 11,
    'Pleural_Thickening': 12,
    'Pneumonia': 13,
    'Pneumothorax': 14
}

In [64]:
for row in patient_df[multi_target][:10]:
    row = [row]
    console.print(mlb.transform(row)[0])

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

[0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]

[0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]

In [69]:
from bitstring import BitArray, Bits

In [74]:
# patient_df[multi_target] = mlb.transform(patient_df[multi_target])
def build_bitstring(
    target: str, df: pd.DataFrame, mlb: MultiLabelBinarizer, inplace=True
):
    logger.info(f"Building bitstring for {target}")
    def __bool_list_to_bitstring(x):
        x = mlb.transform([x])[0]
        x = [BitArray(bin="1" if b == 1 else "0") for b in x]
        x = BitArray().join(x)
        return x

    transform = df[target].apply(lambda x: __bool_list_to_bitstring(x))
    if inplace:
        df[target] = transform
    else:
        return transform


build_bitstring(multi_target, patient_df, mlb, inplace=False)

Image Index
00000001_000.png    [False, True, False, False, False, False, Fals...
00000001_001.png    [False, True, False, False, False, True, False...
00000001_002.png    [False, True, False, False, True, False, False...
00000002_000.png    [False, False, False, False, False, False, Fal...
00000003_000.png    [False, False, False, False, False, False, Fal...
                                          ...                        
00030801_001.png    [False, False, False, False, False, False, Fal...
00030802_000.png    [False, False, False, False, False, False, Fal...
00030803_000.png    [False, False, False, False, False, False, Fal...
00030804_000.png    [False, False, False, False, False, False, Fal...
00030805_000.png    [False, False, False, False, False, False, Fal...
Name: multi_Finding Labels, Length: 112120, dtype: object

### Bitwise operations
Since we now have everything setup as a bitstring, we can use bit operations for subsetting the data rather than the previous way I was doing it. Here's an example:

In [75]:
console.print(all_class_encoding)

{
    'Atelectasis': 0,
    'Cardiomegaly': 1,
    'Consolidation': 2,
    'Edema': 3,
    'Effusion': 4,
    'Emphysema': 5,
    'Fibrosis': 6,
    'Hernia': 7,
    'Infiltration': 8,
    'Mass': 9,
    'No Finding': 10,
    'Nodule': 11,
    'Pleural_Thickening': 12,
    'Pneumonia': 13,
    'Pneumothorax': 14
}

In [76]:
patient_df[multi_target] = build_bitstring(multi_target, patient_df, mlb, inplace=False)

In [77]:
# grab first three rows
a = patient_df.iloc[0]
a

Finding Labels                                               Cardiomegaly
Patient ID                                                              1
Patient Age                                                            58
Patient Gender                                                          M
image_files             /home/nicoleg/workspaces/dissertation/.data/Ch...
multi_Finding Labels    [False, True, False, False, False, False, Fals...
Name: 00000001_000.png, dtype: object

In [92]:
def intlist_to_bitstring(x):
    return BitArray().join([BitArray(bin="1" if b == 1 else "0") for b in x])

In [93]:
# positive-class only bitstring formation

positive_class_encoding = all_class_encoding[positive_class]
_BITSTRING_POSITIVE_CLASS = np.zeros(len(all_class_encoding), dtype=int)
_BITSTRING_POSITIVE_CLASS[positive_class_encoding] = 1
BITSTRING_POSITIVE_CLASS = intlist_to_bitstring(_BITSTRING_POSITIVE_CLASS)
# BITSTRING_INVERSE_POSITIVE_CLASS = np.array([0 if i else 1 for i in BITSTRING_POSITIVE_CLASS])

BITSTRING_POSITIVE_CLASS, mlb.inverse_transform(_BITSTRING_POSITIVE_CLASS.reshape(1, -1))
# BITSTRING_INVERSE_POSITIVE_CLASS

(BitArray('0b000000000000010'), [('Pneumonia',)])

In [94]:
def class_to_bitarray(label: str, label_encoding: dict):
    encoding = label_encoding[label]
    _BITSTRING_CLASS = np.zeros(len(all_class_encoding), dtype=int)
    _BITSTRING_CLASS[encoding] = 1
    BITSTRING_CLASS = intlist_to_bitstring(_BITSTRING_CLASS)

    return BITSTRING_CLASS


pneumonia_bitstring = class_to_bitarray("Pneumonia", all_class_encoding)
pneumonia_bitstring

BitArray('0b000000000000010')

In [95]:
_NULL = np.zeros(len(all_class_encoding), dtype=int)
NULL = intlist_to_bitstring(_NULL)
NULL

BitArray('0b000000000000000')

In [99]:
# checks if an instance contains only the positive class
a = intlist_to_bitstring(a[multi_target])
a & BITSTRING_POSITIVE_CLASS == NULL

True

In [101]:
pneumonia_bitstring & BITSTRING_POSITIVE_CLASS == NULL

False

In [ ]:
def subset_to_positive_class(
    df: pd.DataFrame, target: str, inplace=True, version: float = 1.0
):
    positive_class_encoding = all_class_encoding[positive_class]
    bitstring_positive_class = class_to_bitarray(positive_class, all_class_encoding)


pneumonia_df = subset_to_positive_class(patient_df, multi_target, inplace=False)
pneumonia_df[[target, multi_target]]

In [ ]:
pneumonia_df[multi_target]

### Make into dataset

In [ ]:
# visualize
train_dataset = ImageDataset(
    image_files=train_df["image_files"].tolist(), labels=train_df[target].tolist()
)
train_loader = ThreadDataLoader(dataset=train_dataset, batch_size=1, num_workers=4)
iterator = iter(train_loader)

In [ ]:
from monai.utils import first

scan, label = next(iterator)
scan.shape, label


In [ ]:
scan.squeeze().shape

In [ ]:
# patient_id = scan._meta["filename_or_obj"].split("/")[-1].split(".")[0]
plt.title(f"Label: {label[0]}")
display_scan = scan.squeeze().numpy()
# display_scan = np.transpose(display_scan, (1, 2, 0))
plt.imshow(display_scan, cmap="bone");

# Using `RTK`

In [ ]:
from omegaconf import DictConfig
import logging

# rtk
from rtk import datasets
from rtk.utils import hydra_instantiate

logging.getLogger("rtk.datasets").setLevel(logging.DEBUG)

dataset = datasets.instantiate_image_dataset(
    cfg=cfg, save_metadata=True
)
train_dataset, test_dataset = dataset[0], dataset[1]

In [ ]:
train_dataset[0][0].shape

In [ ]:
train_loader = hydra_instantiate(
    cfg=dataset_cfg.dataloader,
    dataset=train_dataset,
    pin_memory=torch.cuda.is_available(),
    shuffle=True,
)
test_loader = hydra_instantiate(
    cfg=dataset_cfg.dataloader,
    dataset=test_dataset,
    pin_memory=torch.cuda.is_available(),
)
iterator = iter(train_loader)


## Get some samples

In [ ]:
scan, label = next(iterator)
scan.shape, label

In [ ]:
_filename = scan[10]._meta['filename_or_obj'].split('/')[-1]
patient_id = _filename.split('.')[0]
patient_id

In [ ]:
plt.title(f"Patient ID: {patient_id}")
display_scan = scan[0].numpy()
display_scan = np.transpose(display_scan, (1, 2, 0))
plt.imshow(display_scan, cmap="bone");

In [ ]:
scan, label = datasets.visualize_scan(iterator=iterator)

# Pediatrics age overlap

In [ ]:
pneumonia = "Pneumonia"
ped = patient_df[patient_df[target].str.contains(pneumonia)]


ped[ped["Patient Age"] < 18]

In [ ]:
print(f"Number of patients with {pneumonia}:", len(ped))
print(f"Number of patients with {pneumonia} and age < 18:", len(ped[ped["Patient Age"] < 18]))
print(f"Number of patients with {pneumonia} and age < 21:", len(ped[ped["Patient Age"] < 21]))

## Combine Chest XRay and Pediatrics data

In [ ]:
dataset_cfg.additional_datasets

In [ ]:
from rtk.utils import yaml_to_configuration

pediatrics_file_path = dataset_cfg.additional_datasets["dataset_configs"][0]["filepath"]

pediatrics: DatasetConfiguration = yaml_to_configuration(pediatrics_file_path)
pediatrics


In [ ]:
from rtk.datasets import load_chest_xray_dataset

ped_train_dataset, ped_test_dataset, ped_train_metadata, ped_test_metadata = load_chest_xray_dataset(
    cfg=cfg,
    dataset_cfg=pediatrics,
    return_metadata=True,
)
ped_train_metadata_subset = ped_train_metadata[ped_train_metadata["labels"] == 1]
ped_test_metadata_subset = ped_test_metadata[ped_test_metadata["labels"] == 1]
ped_train_metadata_subset

In [ ]:
vars(cfg)

In [ ]:
combined_train_dataset = deepcopy(train_dataset)
combined_test_dataset = deepcopy(test_dataset)
combined_train_dataset.image_files

In [ ]:
combined_train_image_files = np.hstack((ped_train_metadata_subset.image_files.values, train_dataset.image_files))
combined_test_image_files = np.hstack((ped_test_metadata_subset.image_files.values, test_dataset.image_files))
combined_train_labels = np.hstack((ped_train_metadata_subset.labels.values, train_dataset.labels))
combined_test_labels = np.hstack((ped_test_metadata_subset.labels.values, test_dataset.labels))


In [ ]:
from rtk.utils import hydra_instantiate

combined_train_dataset = hydra_instantiate(
    cfg=dataset_cfg.instantiate,
    image_files=combined_train_image_files,
    labels=combined_train_labels,
    transform=train_dataset.transform,
)
combined_test_dataset = hydra_instantiate(
    cfg=dataset_cfg.instantiate,
    image_files=combined_test_image_files,
    labels=combined_test_labels,
    transform=test_dataset.transform,
)
combined_train_dataset

In [ ]:
from collections import Counter

Counter(train_dataset.labels), Counter(combined_train_dataset.labels)

In [ ]:
Counter(test_dataset.labels), Counter(combined_test_dataset.labels)

In [ ]:
from rtk.datasets import combine_datasets

combined_train_dataset, combined_test_dataset = combine_datasets(train_dataset, test_dataset, dataset_cfg=dataset_cfg)
# combined_test_dataset = combine_datasets(test_dataset, ped_test_dataset, dataset_cfg=dataset_cfg, transform=test_dataset.transform)
combined_train_dataset.image_files

In [ ]:
len(combined_train_dataset.image_files)

In [ ]:
combined_train_dataloader = hydra_instantiate(
    cfg=dataset_cfg.dataloader,
    dataset=combined_train_dataset,
    pin_memory=torch.cuda.is_available(),
    shuffle=True,
)
combined_test_dataloader = hydra_instantiate(
    cfg=dataset_cfg.dataloader,
    dataset=combined_test_dataset,
    pin_memory=torch.cuda.is_available(),
    shuffle=True,
)
combined_train_iterator = iter(combined_train_dataloader)
combined_test_iterator = iter(combined_test_dataloader)


In [ ]:
scan, label = datasets.visualize_scan(iterator=combined_train_iterator)

In [ ]:
combined_train_dataset.image_files

## with RTK

In [ ]:
Counter(train_dataset.labels), Counter(test_dataset.labels)


In [ ]:
train_dataset, test_dataset = datasets.combine_datasets(train_dataset, test_dataset, dataset_cfg=dataset_cfg)

Counter(train_dataset.labels), Counter(test_dataset.labels)
